sklearn官方文档：https://scikit-learn.org/stable/index.html  
sklearn中文文档：http://sklearn.apachecn.org/#/

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
%matplotlib inline

# 解决中文显示问题
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False

# 特征选择的目的

维灾难就是当特征维度超过一定界限后，分类器的性能随着特征维度的增加反而下降（而且维度越高训练模型的时间开销也会越大）。导致分类器下降的原因往往是因为这些高纬度特征中含有无关特征和冗余特征，因此特征选择的主要目的是去除特征中的无关特征和冗余特征：  
1、无关特征：是指与当前学习任务无关的特征（该特征所提供的信息对于当前学习任务无用），如对于学生成绩而言，学号则是无关特征。  
2、冗余特征：是指该特征所包含的信息能从其他特征推演出来，如对于“面积”这个特征而言，从能从“长”和“宽”得出，则它是冗余特征。  
特征选择主要有两个功能：  
1、减少特征数量、降维，使模型泛化能力更强，减少过拟合  
2、增强对特征和特征值之间的理解  
通常来说，从两个方面考虑来选择特征：  
1、特征是否发散：如果一个特征不发散，例如方差接近于0，也就是说样本在这个特征上基本上没有差异，这个特征对于样本的区分并没有什么用。  
2、特征与目标的相关性：这点比较显见，与目标相关性高的特征，应当优选选择。除方差法外，本文介绍的其他方法均从相关性考虑。  

# 方差选择法

这应该是最简单的特征选择方法了：假设某特征的特征值只有0和1，并且在所有输入样本中，95%的实例的该特征取值都是1，那就可以认为这个特征作用不大,可以把它作为特征选择的预处理，先去掉那些取值变化小的特征，然后后续的特征选择方法中选择合适的进行进一步的特征选择。  
VarianceThreshold:设置一个方差阈值，没有达到这个方差阈值的特征都会被丢弃。

In [8]:
from sklearn import datasets
data=sns.load_dataset("iris")
data['species'] = data['species'].map(dict(zip(data['species'].unique(),range(data['species'].nunique()))))

# data.var().sort_values(ascending=True)
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
sel.fit_transform(data)

# Pearson相关系数

皮尔森相关系数是一种最简单的，能帮助理解特征和响应变量之间关系的方法，该方法衡量的是变量之间的线性相关性，结果的取值区间为[-1，1]，-1表示完全的负相关(这个变量下降，那个就会上升)，+1表示完全的正相关，0表示没有线性相关。Pearson Correlation速度快、易于计算，经常在拿到数据(经过清洗和特征提取之后的)之后第一时间就执行。

In [19]:
from sklearn import datasets
data=sns.load_dataset("iris")
data['species'] = data['species'].map(dict(zip(data['species'].unique(),range(data['species'].nunique()))))
# np.corrcoef(data['sepal_length'],data['sepal_width'],rowvar=0)
np.corrcoef(data,rowvar=0)
data.corr(method='pearson') # 自动排除类型为字符串的列

"""Pearson相关系数的一个明显缺陷是，作为特征排序机制，他只对线性关系敏感。如果关系是非线性的，
即便两个变量具有一一对应的关系，Pearson相关性也可能会接近0。"""

array([[ 1.        , -0.11756978,  0.87175378,  0.81794113,  0.78256123],
       [-0.11756978,  1.        , -0.4284401 , -0.36612593, -0.42665756],
       [ 0.87175378, -0.4284401 ,  1.        ,  0.96286543,  0.9490347 ],
       [ 0.81794113, -0.36612593,  0.96286543,  1.        ,  0.95654733],
       [ 0.78256123, -0.42665756,  0.9490347 ,  0.95654733,  1.        ]])

,sepal_length,sepal_width,petal_length,petal_width,species
sepal_length,1.000000,-0.117570,0.871754,0.817941,0.782561
sepal_width,-0.117570,1.000000,-0.428440,-0.366126,-0.426658
petal_length,0.871754,-0.428440,1.000000,0.962865,0.949035
petal_width,0.817941,-0.366126,0.962865,1.000000,0.956547
species,0.782561,-0.426658,0.949035,0.956547,1.000000


'Pearson相关系数的一个明显缺陷是，作为特征排序机制，他只对线性关系敏感。如果关系是非线性的，\n即便两个变量具有一一对应的关系，Pearson相关性也可能会接近0。'

# 卡方检验

经典的卡方检验是检验定性自变量对定性因变量的相关性,可以选择排名前k个的特征或排名在前k%的特征,选择的统计指标需要指定，  
对于regression问题，使用f_regression指标;对于classification问题，可以使用chi2或者f_classif指标。

In [1]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
iris = load_iris()
X, y = iris.data, iris.target
X.shape
X_new = SelectKBest(chi2, k=2).fit_transform(X, y)
X_new.shape

(150, 4)

(150, 2)

# 特征选择案例

In [18]:
data = pd.read_csv('data/featurechoose.csv',encoding = 'gb18030')
X=data.values[:,0:3]
y=data.values[:,3]

array([1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 1], dtype=int64)

In [21]:
# 1、利用VarianceThreshold()来删除方差低于阈值的变量
print('原始数据：',X)
vt = VarianceThreshold(threshold=(.8 * (1 - .8)))
Xt = vt.fit_transform(X)
print('各特征方差：',vt.variances_)
print('方差阈值选择后的数据：',Xt) # 这个时候，第1列就被删除了，因为它的方差低于阈值

原始数据： [[1 1 4]
 [1 0 3]
 [1 3 5]
 [1 1 5]
 [1 0 5]
 [1 1 1]
 [1 3 0]
 [1 2 2]
 [1 3 3]
 [1 1 4]
 [2 1 5]
 [1 3 4]
 [1 0 3]
 [1 0 3]
 [1 3 4]
 [1 0 5]
 [1 0 2]
 [1 1 5]
 [1 0 3]
 [1 2 0]
 [1 2 5]
 [1 2 3]
 [1 1 3]
 [1 1 1]
 [1 0 2]
 [1 0 1]
 [1 1 5]
 [1 2 4]
 [2 2 5]
 [1 0 4]
 [1 3 4]
 [1 1 2]
 [1 3 5]
 [2 3 5]
 [1 3 2]
 [1 0 1]
 [1 1 4]
 [1 0 1]
 [1 1 4]
 [1 1 1]
 [1 2 3]
 [1 3 4]
 [1 2 1]
 [2 2 3]
 [1 1 5]
 [1 1 5]
 [1 2 4]
 [1 2 2]]
各特征方差： [ 0.07638889  1.15581597  2.34331597]
方差阈值选择后的数据： [[1 4]
 [0 3]
 [3 5]
 [1 5]
 [0 5]
 [1 1]
 [3 0]
 [2 2]
 [3 3]
 [1 4]
 [1 5]
 [3 4]
 [0 3]
 [0 3]
 [3 4]
 [0 5]
 [0 2]
 [1 5]
 [0 3]
 [2 0]
 [2 5]
 [2 3]
 [1 3]
 [1 1]
 [0 2]
 [0 1]
 [1 5]
 [2 4]
 [2 5]
 [0 4]
 [3 4]
 [1 2]
 [3 5]
 [3 5]
 [3 2]
 [0 1]
 [1 4]
 [0 1]
 [1 4]
 [1 1]
 [2 3]
 [3 4]
 [2 1]
 [2 3]
 [1 5]
 [1 5]
 [2 4]
 [2 2]]


In [22]:
# 2、利用卡方检验选择最佳特征
transformer = SelectKBest(score_func=chi2, k=2)
Xt_chi2 = transformer.fit_transform(X, y)
print('卡方检验各特征得分',transformer.scores_)
print('卡方检验选择后的数据',Xt_chi2) #最佳特征2、3列

卡方检验各特征得分 [ 0.16138489  2.61995828  2.01656812]
卡方检验选择后的数据 [[1 4]
 [0 3]
 [3 5]
 [1 5]
 [0 5]
 [1 1]
 [3 0]
 [2 2]
 [3 3]
 [1 4]
 [1 5]
 [3 4]
 [0 3]
 [0 3]
 [3 4]
 [0 5]
 [0 2]
 [1 5]
 [0 3]
 [2 0]
 [2 5]
 [2 3]
 [1 3]
 [1 1]
 [0 2]
 [0 1]
 [1 5]
 [2 4]
 [2 5]
 [0 4]
 [3 4]
 [1 2]
 [3 5]
 [3 5]
 [3 2]
 [0 1]
 [1 4]
 [0 1]
 [1 4]
 [1 1]
 [2 3]
 [3 4]
 [2 1]
 [2 3]
 [1 5]
 [1 5]
 [2 4]
 [2 2]]


In [23]:
# 3、利用皮尔逊(Pearson)相关系数选择最佳特征：SciPy库的pearsonr()函数
# 定义函数
def multivariate_pearsonr(X, y):
    scores, pvalues = [], []
    for column in range(X.shape[1]): #X.shape[1]返回列数
        cur_score, cur_p = pearsonr(X[:,column], y)
        scores.append(abs(cur_score))
        pvalues.append(cur_p)
    return (np.array(scores), np.array(pvalues))
transformer = SelectKBest(score_func=multivariate_pearsonr, k=2)
Xt_pearson = transformer.fit_transform(X, y)
print('各特征相关系数',transformer.scores_)
print('相关系数选择后的数据',Xt_pearson)

各特征相关系数 [ 0.21836186  0.25674323  0.24061109]
相关系数选择后的数据 [[1 4]
 [0 3]
 [3 5]
 [1 5]
 [0 5]
 [1 1]
 [3 0]
 [2 2]
 [3 3]
 [1 4]
 [1 5]
 [3 4]
 [0 3]
 [0 3]
 [3 4]
 [0 5]
 [0 2]
 [1 5]
 [0 3]
 [2 0]
 [2 5]
 [2 3]
 [1 3]
 [1 1]
 [0 2]
 [0 1]
 [1 5]
 [2 4]
 [2 5]
 [0 4]
 [3 4]
 [1 2]
 [3 5]
 [3 5]
 [3 2]
 [0 1]
 [1 4]
 [0 1]
 [1 4]
 [1 1]
 [2 3]
 [3 4]
 [2 1]
 [2 3]
 [1 5]
 [1 5]
 [2 4]
 [2 2]]


In [26]:
# 4、利用CART分类器，查看特征选择的准确率
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

clf = DecisionTreeClassifier(random_state=14)
scores_chi2 = cross_val_score(clf, Xt_chi2, y, scoring='accuracy')
scores_pearson = cross_val_score(clf, Xt_pearson, y, scoring='accuracy')

print("Chi2 performance: {0:.3f}".format(scores_chi2.mean()))
print("Pearson performance: {0:.3f}".format(scores_pearson.mean()))

Chi2 performance: 0.440
Pearson performance: 0.440
